In [1]:
import json
import io
import numpy as np
import os
import librosa
import pandas as pd
import pickle
import cv2
import pprint
import pandas as pd
# matplotlib for displaying the output
import matplotlib.pyplot as plt
import argparse

import librosa
import librosa.display
import yaml


from collections import OrderedDict


In [2]:
def read(file_path):
    with io.open(file_path, encoding='utf-8-sig') as f:
        #io.open(f, encoding='utf-8-sig')
        #raw_data = yaml.safe_load(f)
        
        raw_data = json.load(f, encoding ='utf-8')
        #aw_data = json.load(f, object_hook=_decode_dict)
    return raw_data



In [3]:
data = read('../sample/KETI_MULTIMODAL_0000000012_interpolation.json')

In [4]:
json_data = data
mp4_path = '../sample/KETI_MULTIMODAL_0000000012.mp4'

In [5]:
#json_data['dialogue_infos']

In [6]:
# def parser(raw_data, mp4_path, parse_save_path): #parsing

cap = cv2.VideoCapture(mp4_path)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS) 
duration = length/fps




dialogue_df = pd.DataFrame()
idx = 0
for i in json_data['dialogue_infos']:
    dialogue = {'speaker_id':i['speaker_id'], 'start_time':i['start_time'], 'end_time':i['end_time'], 'duration':duration}
    data = pd.DataFrame(OrderedDict(dialogue),  index=[idx])
    dialogue_df = dialogue_df.append(data)
    idx += 1



In [7]:

cap = cv2.VideoCapture(mp4_path)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS) 
duration = length/fps

dialogue_df = pd.DataFrame()
idx = 0
for i in json_data['dialogue_infos']:
    dialogue = {'speaker_id':i['speaker_id'], 'start_time':i['start_time'], 'end_time':i['end_time'], 'duration':duration}
    data = pd.DataFrame( OrderedDict(dialogue),index=[idx])
    dialogue_df = dialogue_df.append(data)
    idx += 1

In [8]:
dialogue_df

,speaker_id,start_time,end_time,duration
0,박두식,00:00:14.950,00:00:15.791,16.4164
1,라미란,00:00:00.036,00:00:02.448,16.4164
2,박선호,00:00:02.954,00:00:05.398,16.4164
3,박두식,00:00:05.744,00:00:12.780,16.4164
4,이영애,00:00:13.199,00:00:14.775,16.4164


In [9]:
df = pd.DataFrame()
idx = 0
for i in json_data['shot_infos']:
    if len(i['visual_infos'][0]['persons']) <= 0:
        continue

    speaker = i['visual_infos'][0]['persons'][0]['person_id']
    frames = []
    for j in i['visual_infos']:
        frames.append(j['frame_id'])
        emotion = j['persons'][0]['person_info']['emotion']
    emotion.update({'frame_id':j['frame_id'], 'frame_length':length, 'person_id':speaker, 'first_frame':min(frames), 'last_frame':max(frames)})
    data = pd.DataFrame(OrderedDict(emotion), index=[idx])
    df = df.append(data)
    idx += 1


In [10]:
df

,happiness,sadness,anger,surprise,afraid,contempt,disgust,neutral,frame_id,frame_length,person_id,first_frame,last_frame
0,10,0,0,0,0,0,0,0,446,492,라미란,401,446
1,10,0,0,0,0,0,0,0,399,492,박두식,367,399
2,10,0,0,0,0,0,0,0,365,492,라미란,328,365
3,10,0,0,0,0,0,0,0,326,492,박선호,265,326
4,0,0,0,0,0,0,0,10,263,492,박두식,213,263
5,10,0,0,0,0,0,0,0,211,492,라미란,172,211
6,10,0,0,0,0,0,0,0,170,492,박선호,92,170
7,10,0,0,0,0,0,0,0,90,492,라미란,41,90
8,10,0,0,0,0,0,0,0,39,492,박선호,9,39


In [11]:
parse_save_path = 'processed'

In [12]:

# #pickle 로 저장
# #log S 와 비교해서 list, array 중에 format 결정할것.
npy_save = np.array(df)
# with open(parse_save_path, 'wb', pickle.HIGHEST_PROTOCOL) as wow:
#     pickle.dump(df, wow)

npy_save_dialogue = np.array(dialogue_df)

# #return npy_save, npy_save_dialogue
# npy_save = df
# npy_save_dialogue=dialogue_df


In [13]:
npy_save

array([[10, 0, 0, 0, 0, 0, 0, 0, 446, 492, '라미란', 401, 446],
       [10, 0, 0, 0, 0, 0, 0, 0, 399, 492, '박두식', 367, 399],
       [10, 0, 0, 0, 0, 0, 0, 0, 365, 492, '라미란', 328, 365],
       [10, 0, 0, 0, 0, 0, 0, 0, 326, 492, '박선호', 265, 326],
       [0, 0, 0, 0, 0, 0, 0, 10, 263, 492, '박두식', 213, 263],
       [10, 0, 0, 0, 0, 0, 0, 0, 211, 492, '라미란', 172, 211],
       [10, 0, 0, 0, 0, 0, 0, 0, 170, 492, '박선호', 92, 170],
       [10, 0, 0, 0, 0, 0, 0, 0, 90, 492, '라미란', 41, 90],
       [10, 0, 0, 0, 0, 0, 0, 0, 39, 492, '박선호', 9, 39]], dtype=object)

In [14]:
npy_save_dialogue

array([['박두식', '00:00:14.950', '00:00:15.791', 16.4164],
       ['라미란', '00:00:00.036', '00:00:02.448', 16.4164],
       ['박선호', '00:00:02.954', '00:00:05.398', 16.4164],
       ['박두식', '00:00:05.744', '00:00:12.780', 16.4164],
       ['이영애', '00:00:13.199', '00:00:14.775', 16.4164]], dtype=object)

In [15]:
audio_name = mp4_path

In [16]:
#{frame_id:, emotion}
#     def split_by_emo(data_path, save_path, npy_save, npy_save_dialogue):
        #data path : mp4 등 저장된 위치
        #save path : 결과물 저장될 위치
        #npy_save parser 로 부터 넘겨받는 데이터
        
#         file_list = os.listdir(data_path)
#         file_list_json = [file for file in file_list if file.endswith(".json")]


#         real_file_name = file_list_json[:-5]  #327.json -> 327 #NEEDS change based on datasets
#         audio_name = real_file_name + '.mp4'
y, sr = librosa.load(audio_name)

label = []
all_arr = []

'''
TODO we will change this (see next line)
'''
# emo = npy_save['emotion'] #TODO 
# all_real_fr_num = npy_save['frame_length'] # 정현님이 주시는 전체 프레임수
# frame_info_s = npy_save['first_frame']
# frame_info_e = npy_save['last_frame']

# fract = len(y)/all_real_fr_num # 프레임수(이미지)에 대한 y 의 비율
# fract2 = len(y)/all_real_time_num # 시간에 대한 y 의 비율


# all_real_time_num = npy_save_dialogue['duration']
# time_info_s = npy_save_dialogue['start_time']
# time_info_e = npy_save_dialogue['end_time']





/home/ksjbang/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


'\nTODO we will change this (see next line)\n'

In [17]:
'''
change into this 

'''
emo = np.argmax(npy_save[:,:8],axis=1)
emo 
#각 문장별로 제일 큰 emotion ... 5번째 문장의 경우는 7번째index(neutral) 감정이다. 
#happiness	sadness	anger	surprise	afraid	contempt	disgust	neutral 

array([0, 0, 0, 0, 7, 0, 0, 0, 0])

In [18]:
all_real_fr_num = npy_save[:,9]
all_real_fr_num


array([492, 492, 492, 492, 492, 492, 492, 492, 492], dtype=object)

In [19]:
frame_info_s = npy_save[:,11]
frame_info_e = npy_save[:,12]
frame_info_s,frame_info_e


(array([401, 367, 328, 265, 213, 172, 92, 41, 9], dtype=object),
 array([446, 399, 365, 326, 263, 211, 170, 90, 39], dtype=object))

In [20]:
all_real_time_num = npy_save_dialogue[:,3]
all_real_time_num

array([16.4164, 16.4164, 16.4164, 16.4164, 16.4164], dtype=object)

In [21]:
fract = len(y)/all_real_fr_num # 프레임수(이미지)에 대한 y 의 비율
fract2 = len(y)/all_real_time_num # 시간에 대한 y 의 비율
fract,fract2

(array([736.1951219512196, 736.1951219512196, 736.1951219512196,
        736.1951219512196, 736.1951219512196, 736.1951219512196,
        736.1951219512196, 736.1951219512196, 736.1951219512196],
       dtype=object),
 array([22063.78986866792, 22063.78986866792, 22063.78986866792,
        22063.78986866792, 22063.78986866792], dtype=object))

In [22]:
npy_save_dialogue

array([['박두식', '00:00:14.950', '00:00:15.791', 16.4164],
       ['라미란', '00:00:00.036', '00:00:02.448', 16.4164],
       ['박선호', '00:00:02.954', '00:00:05.398', 16.4164],
       ['박두식', '00:00:05.744', '00:00:12.780', 16.4164],
       ['이영애', '00:00:13.199', '00:00:14.775', 16.4164]], dtype=object)

In [23]:
def str_to_sec(str_list):
    temp = []
    for str_val in str_list:
        if isinstance(str_val, str):
            h, m, s = str_val.split(':')
            temp.append(int(h) * 3600 + int(m) * 60 +int(float(s)))
                        
        else:
            print(str_val)
            
    return temp

time_info_s = str_to_sec(npy_save_dialogue[:,1])
time_info_e = str_to_sec(npy_save_dialogue[:,2])

time_info_s, time_info_e

([14, 0, 2, 5, 13], [15, 2, 5, 12, 14])

In [24]:

'''
TODO 
label will be just expanded version of emo 
'''
# for k in range(0, label.shape[0]):
#     now_e = emo[k]   #emotion 라 dic key emotion
#     now_ss = frame_info_s[k]*fract
#     now_es = frame_info_e[k]*fract
#     now_ss_2 = time_info_s[k]*fract2
#     now_es_2 = time_info_e[k]*fract2

#     if now_ss >= now_ss_2 :
#         if now_es <= now_es_2 :

#             # 0=anger, 1=disgust&contempt, 2=afraid, 3=happiness, 4=sadness, 5=surprise, 6=neutral

#             if (now_e == 'anger'):
#                 label.append(0)
#                 now_l = 0
#             if (now_e == 'disgust'):
#                 label.append(1)
#                 now_l = 1
#             if (now_e == 'contempt'):
#                 label.append(1)
#                 now_l = 1
#             if (now_e == 'afraid'):
#                 label.append(2)
#                 now_l = 2
#             if (now_e == 'happiness'):
#                 label.append(3)
#                 now_l = 3
#             if (now_e == 'sadness'):
#                 label.append(4)
#                 now_l = 4
#             if (now_e == 'surprise'):
#                 label.append(5)
#                 now_l = 5
#             if (now_e == 'neutral'):
#                 label.append(6)
#                 now_l = 6

#             all_arr.append(y[now_ss:now_es])
#             S = librosa.feature.melspectrogram(np.asarray(y[now_ss:now_es]), sr=sr, n_mels=128)

#             # Convert to log scale (dB). We'll use the peak power (max) as reference.
#             # Check the Log S format
#             log_S = librosa.power_to_db(S, ref=np.max)

#             # 여기서 라벨에 따라서 save path 를 변경
#             this_save_path = os.path.join(save_path,now_l,real_file_name+'_'+time_info_s[k]+ '_' + time_info_e[k] + '.png') #save to datasets/0/KETI_MULTIMODAL_0000000012_334_235345

#             plt.switch_backend('agg')
#             # Make a new figure
#             plt.figure(figsize=(12, 4))

#             # Display the spectrogram on a mel scale
#             # sample rate and hop length parameters are used to render the time axis
#             librosa.display.specshow(log_S, sr=sr, x_axis='off', y_axis='off')

#             # Put a descriptive title on the plot
#             # plt.title('mel power spectrogram')

#             # draw a color bar
#             # plt.colorbar(format='%+02.0f dB')

#             # Make the figure layout compact
#             plt.tight_layout()
#             plt.savefig(this_save_path)

#return log_S, label

'\nTODO \nlabel will be just expanded version of emo \n'

In [25]:
npy_save

array([[10, 0, 0, 0, 0, 0, 0, 0, 446, 492, '라미란', 401, 446],
       [10, 0, 0, 0, 0, 0, 0, 0, 399, 492, '박두식', 367, 399],
       [10, 0, 0, 0, 0, 0, 0, 0, 365, 492, '라미란', 328, 365],
       [10, 0, 0, 0, 0, 0, 0, 0, 326, 492, '박선호', 265, 326],
       [0, 0, 0, 0, 0, 0, 0, 10, 263, 492, '박두식', 213, 263],
       [10, 0, 0, 0, 0, 0, 0, 0, 211, 492, '라미란', 172, 211],
       [10, 0, 0, 0, 0, 0, 0, 0, 170, 492, '박선호', 92, 170],
       [10, 0, 0, 0, 0, 0, 0, 0, 90, 492, '라미란', 41, 90],
       [10, 0, 0, 0, 0, 0, 0, 0, 39, 492, '박선호', 9, 39]], dtype=object)

In [26]:
# time_info_s[1],fract2

In [27]:
# time_info_s[1]*fract2[0]
emo

array([0, 0, 0, 0, 7, 0, 0, 0, 0])

In [42]:

audio_part = mp4_path.split('_')[2]
audio_part_name=audio_part.split('.')[0]


In [57]:
for i in range(0,8):
    print(i)
    if not os.path.exists('../preprocessed/'+str(now_e)):
        os.makedirs('../preprocessed/'+str(now_e))


0
1
2
3
4
5
6
7


In [56]:

''' 
TODO (working...) 
label will be just expanded version of emo 
'''
# for k in range(0, label.shape[0]):

for k in range(0, npy_save.shape[0]): #changed
    try:
        now_e = emo[k]   #emotion 라 dic key emotion
        now_ss = frame_info_s[k]*fract
        now_es = frame_info_e[k]*fract


        '''
        error because of time mismatch  TODO
        '''
        #now_ss_2 = time_info_s[k]*fract2
        now_ss_2 = time_info_s[k]*fract2[0]  
        #now_es_2 = time_info_e[k]*fract2
        #print(time_info_e[k])
        now_es_2 = time_info_e[k]*fract2[0]



        #     if now_ss >= now_ss_2 :
        #         if now_es <= now_es_2 :

            # 0=anger, 1=disgust&contempt, 2=afraid, 3=happiness, 4=sadness, 5=surprise, 6=neutral

        #             if (now_e == 'anger'):
        #                 label.append(0)
        #                 now_l = 0
        #             if (now_e == 'disgust'):
        #                 label.append(1)
        #                 now_l = 1
        #             if (now_e == 'contempt'):
        #                 label.append(1)
        #                 now_l = 1
        #             if (now_e == 'afraid'):
        #                 label.append(2)
        #                 now_l = 2
        #             if (now_e == 'happiness'):
        #                 label.append(3)
        #                 now_l = 3
        #             if (now_e == 'sadness'):
        #                 label.append(4)
        #                 now_l = 4
        #             if (now_e == 'surprise'):
        #                 label.append(5)
        #                 now_l = 5
        #             if (now_e == 'neutral'):
        #                 label.append(6)
        #                 now_l = 6
        label.append(now_e) 
        #print(label)


        all_arr.append(y[int(now_ss[0]):int(now_es[0])])
        S = librosa.feature.melspectrogram(np.asarray(y[int(now_ss[0]):int(now_es[0])]), sr=sr, n_mels=128)

        # Convert to log scale (dB). We'll use the peak power (max) as reference.
        # Check the Log S format
        log_S = librosa.power_to_db(S, ref=np.max)

        # 여기서 라벨에 따라서 save path 를 변경
        #save_path = 'samples/emo'
        #this_save_path = os.path.join(save_path,now_e,real_file_name+'_'+time_info_s[k]+ '_' + time_info_e[k] + '.png') #save to datasets/0/KETI_MULTIMODAL_0000000012_334_235345

        plt.switch_backend('agg')
        # matplotlib = reload(matplotlib)
        # Make a new figure
        plt.figure(figsize=(12, 4))

        # Display the spectrogram on a mel scale
        # sample rate and hop length parameters are used to render the time axis
        librosa.display.specshow(log_S, sr=sr, x_axis='off', y_axis='off')

        # Put a descriptive title on the plot
        # plt.title('mel power spectrogram')


        # draw a color bar
        # plt.colorbar(format='%+02.0f dB')

        # Make the figure layout compact
        plt.tight_layout()
        path=os.path.join('../preprocessed',str(now_e),audio_part_name+'_'+str(k)+'.png')
        plt.savefig(path)
        print('success',path)
    except : 
        print("errrr",k)

#return log_S, label

success ../preprocessed/0/0000000012_0.png
success ../preprocessed/0/0000000012_1.png
success ../preprocessed/0/0000000012_2.png
success ../preprocessed/0/0000000012_3.png
success ../preprocessed/7/0000000012_4.png
errrr 5
errrr 6
errrr 7
errrr 8
